# Introduction

This tutorial illustrates how to use an *ObjTables* schema to model data, as well as how to use *ObjTables* to query, edit, compare, normalize, and validate datasets. The tutorial uses an address book of CEOs as an example.

# Define a schema for an address book

First, as described in [Tutorial 1](1.%20Building%20and%20visualizing%20schemas.ipynb), use *ObjTables* to define a schema for an address book.

In [1]:
import enum
import obj_tables


class Address(obj_tables.Model):
    street = obj_tables.StringAttribute(unique=True, primary=True, verbose_name='Street')
    city = obj_tables.StringAttribute(verbose_name='City')
    state = obj_tables.StringAttribute(verbose_name='State')
    zip_code = obj_tables.StringAttribute(verbose_name='Zip code')
    country = obj_tables.StringAttribute(verbose_name='Country')

    class Meta(obj_tables.Model.Meta):
        table_format = obj_tables.TableFormat.multiple_cells
        attribute_order = ('street', 'city', 'state', 'zip_code', 'country',)
        verbose_name = 'Address'
        verbose_name_plural = 'Addresses'
        

class Company(obj_tables.Model):
    name = obj_tables.StringAttribute(unique=True, primary=True, verbose_name='Name')
    url = obj_tables.UrlAttribute(verbose_name='URL')
    address = obj_tables.OneToOneAttribute(Address, related_name='company', verbose_name='Address')

    class Meta(obj_tables.Model.Meta):
        table_format = obj_tables.TableFormat.column
        attribute_order = ('name', 'url', 'address',)
        verbose_name = 'Company'
        verbose_name_plural = 'Companies'


class PersonType(str, enum.Enum):
    family = 'family'
    friend = 'friend'
    business = 'business'


class Person(obj_tables.Model):
    name = obj_tables.StringAttribute(unique=True, primary=True, verbose_name='Name')
    type = obj_tables.EnumAttribute(PersonType, verbose_name='Type')
    company = obj_tables.ManyToOneAttribute(Company, related_name='employees', verbose_name='Company')
    email_address = obj_tables.EmailAttribute(verbose_name='Email address')
    phone_number = obj_tables.StringAttribute(verbose_name='Phone number')
    address = obj_tables.ManyToOneAttribute(Address, related_name='people', verbose_name='Address')

    class Meta(obj_tables.Model.Meta):
        table_format = obj_tables.TableFormat.row
        attribute_order = ('name', 'type', 'company', 'email_address', 'phone_number', 'address',)
        verbose_name = 'Person'
        verbose_name_plural = 'People'

# Use the address book schema to build an address book of technology companies and their CEOs

Second, create instances of `Company` and `Person` to represent several major technology companies and their CEOs.

##### Create objects to represent several major technology companies and their CEOs

In [2]:
# Tim Cook of Apple
apple = Company(name='Apple',
                url='https://www.apple.com/',
                address=Address(street='10600 N Tantau Ave',
                                city='Cupertino',
                                state='CA',
                                zip_code='95014',
                                country='US'))
cook = Person(name='Tim Cook',
              type=PersonType.business,
              company=apple,
              email_address='tcook@apple.com',
              phone_number='408-996-1010',
              address=apple.address)

# Reed Hasting of Netflix
netflix = Company(name='Netflix',
                  url='https://www.netflix.com/',
                  address=Address(street='100 Winchester Cir',
                                  city='Los Gatos',
                                  state='CA',
                                  zip_code='95032',
                                  country='US'))
hastings = Person(name='Reed Hastings',
                  type=PersonType.business,
                  company=netflix,
                  email_address='reed.hastings@netflix.com',
                  phone_number='408-540-3700',
                  address=netflix.address)

# Sundar Pichai of Google
google = Company(name='Google',
                 url='https://www.google.com/',
                 address=Address(street='1600 Amphitheatre Pkwy',
                                 city='Mountain View',
                                 state='CA',
                                 zip_code='94043',
                                 country='US'))
pichai = Person(name='Sundar Pichai',
                type=PersonType.business,
                company=google,
                email_address='sundar@google.com',
                phone_number='650-253-0000',
                address=google.address)

# Mark Zuckerberg of Facebook
facebook = Company(name='Facebook',
                   url='https://www.facebook.com/',
                   address=Address(street='1 Hacker Way #15',
                                   city='Menlo Park',
                                   state='CA',
                                   zip_code='94025',
                                   country='US'))
zuckerberg = Person(name='Mark Zuckerberg',
                    type=PersonType.business,
                    company=facebook,
                    email_address='zuck@fb.com',
                    phone_number='650-543-4800',
                    address=facebook.address)

##### Merge the companies and CEOs into an address book

In [3]:
companies = [apple, facebook, google, netflix]
ceos = [cook, zuckerberg, pichai, hastings]
address_book = companies + ceos

# Access and edit the properties of the companies and their CEOS

Next, read and write the attributes of instances of `Company` and `Person` to view and edit the part of the address book.

##### Get the all of the employees of Facebook

In [4]:
facebook.employees

##### Get all of the employees who work at Facebook's headquarters

In [5]:
facebook.employees.get(address=facebook.address)

##### Get the employee whose email is `zuck@fb.com`

In [6]:
facebook.employees.get_one(email_address='zuck@fb.com')

##### Set Facebook's URL to its corporate website

In [7]:
facebook.url = 'https://about.fb.com/'

# Compare companies and their CEOs, and find their differences

Next, use the `is_equal` and `difference` methods to compare companies and their CEOs and find their differences.

###### Create a copy of Mark Zuckerberg and check that its equal to the original

In [8]:
zuckerberg_copy = Person(name='Mark Zuckerberg',
                    type=PersonType.business,
                    company=facebook,
                    email_address='zuck@fb.com',
                    phone_number='650-543-4800',
                    address=facebook.address)
print(zuckerberg_copy.is_equal(zuckerberg))

True


###### Add a entry for the COO of Facebook, and find the differences between her and the CEO

In [9]:
sandberg = Person(name='Sheryl Sandberg',
                    type=PersonType.business,
                    company=facebook,
                    email_address='sheryl@fb.com',
                    phone_number='650-543-4800',
                    address=facebook.address)
assert not sandberg.is_equal(zuckerberg)
print(sandberg.difference(zuckerberg))

Objects (Person: "Sheryl Sandberg", Person: "Mark Zuckerberg") have different attribute values:
  `email_address` are not equal:
    sheryl@fb.com != zuck@fb.com
  `name` are not equal:
    Sheryl Sandberg != Mark Zuckerberg


# Sort (normalize) the address book into a reproducible order

In this example, the order of the people in `Company.employees` has no semantic meaning. For example, two instances of `Company` would be considered equal if even the orders of the people in their `employees` attributes were different.

To reproducibly conduct computations on `Company.employees`, irrespective of its order, first use the `normalize` method to sort the relationships between objects into reproducible orders.

##### Define the employees of Facebook in two different orders, sort them, and check that the orders are reproducible

In [10]:
facebook.employees = [zuckerberg, sandberg]
facebook.normalize()
employees_names_a = list(e.name for e in facebook.employees)

facebook.employees = [sandberg, zuckerberg]
facebook.normalize()
employees_names_b = list(e.name for e in facebook.employees)

assert employees_names_a == employees_names_b

# Validate that the technology companies and their CEOs are a valid address book and find any errors

Lastly, use `obj_tables.Validator` to validate the address book of technology companies and their CEOs, and find any errors.

###### Verify that the address bool is valid

In [11]:
errors = obj_tables.Validator().run(address_book)
assert errors is None

###### Add the copy of Mark Zuckerberg to the address book, and verify that the address book is no longer valid because the name `Mark Zuckerberg` is no longer unique

In [12]:
address_book.append(zuckerberg_copy)
errors = obj_tables.Validator().run(address_book)
assert errors is not None
print(errors)

Person:
  'name':
    name values must be unique, but these values are repeated: 'Mark Zuckerberg'
